In [1]:
from music21 import converter,instrument,note,chord,stream
import numpy as np
import pickle

In [2]:
from keras.models import load_model

Using TensorFlow backend.


In [3]:
model = load_model("piano_model.hdf5")

In [4]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 50, 512)           1052672   
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 512)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 50, 512)           2099200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 50, 512)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)              

In [5]:
with open("Notes" , 'rb') as f:
    notes = pickle.load(f)

In [6]:
print("Total Elements : ",len(notes))
print("Unique Elements : ",len(set(notes)))

Total Elements :  117989
Unique Elements :  652


In [7]:
pitchnames = sorted(set(notes))

In [8]:
element_to_label = dict((ele,num) for num,ele in enumerate(pitchnames))

In [9]:
print(element_to_label['F#4'])

628


In [10]:
sequence_length = 50

In [11]:
label_to_element = dict((value,key) for (key,value) in element_to_label.items())

In [12]:
print(label_to_element[142])

10+11+3+6


In [13]:
    network_input = []
    for i in range(len(notes)-sequence_length):
        seq_in = notes[i:i+sequence_length]    
        network_input.append([element_to_label[ch] for ch in seq_in])

In [14]:
print(network_input[350])

[648, 603, 640, 585, 607, 591, 614, 585, 642, 590, 615, 603, 641, 648, 630, 640, 614, 585, 640, 642, 590, 603, 616, 622, 620, 621, 609, 597, 609, 597, 589, 629, 643, 616, 597, 603, 620, 597, 155, 642, 400, 615, 603, 641, 630, 640, 642, 615, 615, 585]


In [15]:
def resultModel(network_input):
    Xtrain = np.reshape(network_input,(-1,len(network_input),1))
    Xtrain = Xtrain/float(len(set(notes)))
    prediction = model.predict(Xtrain)
    result = np.argmax(prediction)
    return result

In [16]:
gen_seq = []

start = np.random.randint(len(network_input)-1)
pattern = network_input[start]
for i in range(100):
    result = resultModel(pattern)
    #print(result)
    gen_seq.append(label_to_element[result])
    pattern.append(result)
    pattern = pattern[1:]

In [17]:
print(gen_seq)

['A4', 'A4', 'F2', 'C4', 'C4', 'F2', 'F2', '0+5', '0+4', 'F4', 'F2', 'E2', 'A3', 'F2', 'C#6', 'E2', 'F2', 'C4', 'C4', 'C4', 'G5', 'F4', 'F4', 'F2', 'A3', 'C3', 'F3', '4+5', '0+5', 'E3', 'E2', 'C4', 'F4', 'F4', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'F5', '11+1', 'F#2', 'B4', 'C5', 'D3', '7+11', 'C6', 'D3', '6+7', '4+7', 'G5', 'C6', 'C4', 'G4', '4+7', '9+0', '4+7', 'G3', 'F2', 'C3', 'E4', 'B2', 'F#3', 'F#5', 'A5', '4+5', 'G2', 'G2', 'G#4', 'G3', 'F#2', 'F#2', 'G3', 'F#2', 'F#2', '0+5', 'G#4', 'B4', '3+5', 'B4', 'B4', 'E6', 'F#4', 'B3', '3+5', 'G4', 'F3', 'D5', '3+5', 'F2', 'E-5', 'E6', 'E-5', 'B4', 'F2', 'D3']


# Generating MIDI from Output

In [18]:
offset = 0
output_notes = []

for pattern in gen_seq:
    
    # if pattern is a chord
    if ('+' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('+')
        temp_notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            temp_notes.append(new_note)
            
        new_chord = chord.Chord(temp_notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
        
    # if pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
        
    if offset < 7:    
        offset += 1.0
    elif offset >47:
        offset += 1.0
    else:
        offset +=0.5

In [19]:
print(offset)

59.5


In [20]:
output_notes

[<music21.note.Note A>,
 <music21.note.Note A>,
 <music21.note.Note F>,
 <music21.note.Note C>,
 <music21.note.Note C>,
 <music21.note.Note F>,
 <music21.note.Note F>,
 <music21.chord.Chord C F>,
 <music21.chord.Chord C E>,
 <music21.note.Note F>,
 <music21.note.Note F>,
 <music21.note.Note E>,
 <music21.note.Note A>,
 <music21.note.Note F>,
 <music21.note.Note C#>,
 <music21.note.Note E>,
 <music21.note.Note F>,
 <music21.note.Note C>,
 <music21.note.Note C>,
 <music21.note.Note C>,
 <music21.note.Note G>,
 <music21.note.Note F>,
 <music21.note.Note F>,
 <music21.note.Note F>,
 <music21.note.Note A>,
 <music21.note.Note C>,
 <music21.note.Note F>,
 <music21.chord.Chord E F>,
 <music21.chord.Chord C F>,
 <music21.note.Note E>,
 <music21.note.Note E>,
 <music21.note.Note C>,
 <music21.note.Note F>,
 <music21.note.Note F>,
 <music21.note.Note C>,
 <music21.note.Note C>,
 <music21.note.Note C>,
 <music21.note.Note C>,
 <music21.note.Note C>,
 <music21.note.Note C>,
 <music21.note.Note C>,

In [21]:
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp = "Output.mid")

'Output.mid'

In [22]:
midi_stream.show('midi')